# High Resolution inference 

In [2]:
# run in the root of the repository
%load_ext autoreload
%autoreload 2
 
%cd ../..

/private/home/pfz/09-videoseal/videoseal-dev


/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
from videoseal.utils.display import save_vid
from videoseal.utils import Timer
from videoseal.evals.full import setup_model_from_checkpoint
from videoseal.evals.metrics import bit_accuracy, pvalue, capacity, psnr, ssim, msssim, linf
from videoseal.data.datasets import VideoDataset
from videoseal.augmentation import Identity, H264, Crop
from videoseal.modules.jnd import JND, JNDSimplified

import os
from tqdm import tqdm
import torch
import gc

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cuda" 

In [ ]:
import os
from tqdm import tqdm
import torch

# Directory containing videos
video_dir = "/checkpoint/pfz/projects/videoseal/assets/videos/metamoviegen_3s"
base_output_folder = "outputs"
if not os.path.exists(base_output_folder):
    os.makedirs(base_output_folder)

# Example usage
ckpts = {
    # "hidden": '/private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.0_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=hidden/checkpoint.pth',
    # "unet": '/private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.5_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=unet_small2/checkpoint.pth',
    # "scaling_laws_smalldetector_tinyembedder":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1105-videoseal0.2-scalinglaws/_lambda_d=0.0_extractor_model=sam_small_embedder_model=0/checkpoint.pth",
    # "scaling_laws_tinydetector_tinyembedder":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1105-videoseal0.2-scalinglaws/_lambda_d=0.0_extractor_model=sam_tiny_embedder_model=0/checkpoint.pth",
    # "JND_fix_discloss":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1108-videoseal0.2-discloss-fix-removeunused-params/_attenuation=jnd_3_3_nbits=64_lambda_d=0.5_video_start=100/checkpoint.pth",
    # "1111_discloss_sleepwake_highssim":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1109-videoseal0.2-discloss-fix-hing-sleepwake-4nodes/_scaling_w=0.1_lambda_i=0.25_disc_hinge_on_logits_fake=False_sleepwake=True_video_start=500/checkpoint.pth"
    # "1111-finetuned":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1111-videoseal0.2-sleepwake-resume/_attenuation=jnd_1_1/checkpoint.pth"
    # "1112-videoseal0.2":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1111-videoseal0.2-archsearch-4nodes/_attenuation=None_nbits=64_finetune_detector_start=1000_embedder_model=unet_small2_quant/checkpoint.pth",
    # "1118-yuv-400":"/checkpoint/pfz/2024_logs/1118_vseal_long_sab/_scheduler=0_optimizer=adopt,lr=1e-5/checkpoint400.pth",
    # "1118-yuv-800":"/checkpoint/pfz/2024_logs/1118_vseal_long_sab/_scheduler=1_optimizer=AdamW,lr=1e-5/checkpoint.pth",
    # "trustmark": "baseline/trustmark",
    # "videoseal0.2b": "/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1111-videoseal0.2-archsearch-4nodes/_attenuation=None_nbits=64_finetune_detector_start=800_embedder_model=unet_small2_quant/checkpoint.pth",
    # "videoseal0.2a": "/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1109-videoseal0.2-discloss-fix-hing-sleepwake-4nodes/_scaling_w=0.5_lambda_i=0.5_disc_hinge_on_logits_fake=True_sleepwake=False_video_start=500/checkpoint.pth",
    # "videoseal0.4": "/large_experiments/meres/hadyelsahar/2024_logs/1120-videoseal0.4/_scaling_w=0.5_sleepwake=False_videowam_step_size=4_extractor_model=sam_tiny/checkpoint.pth",
    # "wam": "baseline/wam",
    # "mbrs": "baseline/mbrs",
    # "128b_y": "/checkpoint/pfz/2025_logs/0219_vseal_convnextextractor/_nbits=128_lambda_i=0.1_embedder_model=1/checkpoint600.pth",
    # # "128b_rgb": "/checkpoint/pfz/2025_logs/0219_vseal_convnextextractor/_nbits=128_lambda_i=0.1_embedder_model=0/checkpoint600.pth",
    # "128b_y_ft": "/checkpoint/pfz/2025_logs/0224_vseal_128bits_convnext_ftvid/_scaling_w=0.3_attenuation=jnd_1_1_augmentation_config=2/checkpoint.pth",
    "128b_y_400": "/checkpoint/pfz/2025_logs/0219_vseal_convnextextractor/_nbits=128_lambda_i=0.1_embedder_model=1/checkpoint400.pth",
}

fps = 24 // 1
# frames_per_clip = fps * 3  # 3s
# frame_step = 1

# a timer to measure the time
timer = Timer()

# Iterate over all checkpoints
for model_name, ckpt in ckpts.items():
    wam = setup_model_from_checkpoint(ckpt)
    wam.eval()
    wam.to(device)

    # wam.blender.scaling_w = 0.012

    scaling_w = 0.8
    attenuation = JND(in_channels=1, out_channels=1, blue=False)
    wam.attenuation = attenuation
    wam.blender.scaling_w = scaling_w

    # scaling_w = 1.0
    # attenuation = JNDSimplified(in_channels=1, out_channels=3, blue=True)
    # wam.attenuation = attenuation
    # wam.blender.scaling_w = scaling_w

    # wam.chunk_size = 200
    wam.step_size = 4

    # Iterate over all video files in the directory
    video_files = [f for f in os.listdir(video_dir) if f.endswith(".mp4")][:1]

    for video_file in tqdm(video_files, desc=f"Processing Videos for {model_name}"):
        video_path = os.path.join(video_dir, video_file)
        base_name = os.path.splitext(video_file)[0]

        # Load video (assuming a function `load_video` exists)
        timer.start()
        vid, mask = VideoDataset.load_full_video_decord(video_path)
        print(f"loading video {video_path} - took {timer.stop():.2f}s")

        # Watermark embedding
        timer.start()
        outputs = wam.embed_lowres_attenuation(vid, is_video=True)
        print(f"embedding watermark  - took {timer.stop():.2f}s")

        # compute diff
        imgs = vid  # b c h w
        imgs_w = outputs["imgs_w"]  # b c h w
        msgs = outputs["msgs"]  # b k
        diff = imgs_w - imgs

        # save
        timer.start()
        save_vid(imgs, f"{base_output_folder}/{model_name}_{base_name}_ori.mp4", fps)
        save_vid(imgs_w, f"{base_output_folder}/{model_name}_{base_name}_wm.mp4", fps)
        save_vid(10*diff.abs(), f"{base_output_folder}/{model_name}_{base_name}_diff.mp4", fps)

        # Metrics
        metrics = {
            "psnr": psnr(imgs, imgs_w, is_video=True).mean().item(),
            "ssim": ssim(imgs, imgs_w).mean().item(),
            "msssim": msssim(imgs, imgs_w).mean().item(),
            "linf": linf(imgs, imgs_w).mean().item()
        }

        # Augment video
        print(f"compressing and detecting watermarks")
        for ii in range(2):
            if ii == 0:
                imgs_aug = imgs_w
                label = "Original"
            if ii == 1: 
                imgs_aug, _ = H264()(imgs_w, crf=30)
                imgs_aug, _ = Crop()(imgs_aug, size=0.75)
                label = "H264 30 + Crop 0.8"

            # detect
            timer.start()
            aggregate = True
            if not aggregate:
                outputs = wam.detect(imgs_aug, is_video=True)
                preds = outputs["preds"]
                bit_preds = preds[:, 1:]  # b k ...
                bit_accuracy_ = bit_accuracy(
                    bit_preds,
                    msgs
                ).nanmean().item()
                metrics[f"bit_accuracy_{label.lower().replace(' ', '_').replace('.', '')}"] = bit_accuracy_
                # print(f"{label} - Bit Accuracy: {bit_accuracy_:.3f} - took {timer.stop():.2f}s")
            else:
                bit_preds = wam.detect_and_aggregate(imgs_aug)
                bit_accuracy_ = bit_accuracy(
                    bit_preds,
                    msgs[:1]
                ).nanmean().item()
                pvalue_ = pvalue(
                    bit_preds,
                    msgs[:1]
                ).nanmean().item()
                capacity_ = capacity(
                    bit_preds,
                    msgs[:1]
                ).nanmean().item()
                metrics[f"bit_accuracy_{label.lower().replace(' ', '_').replace('.', '')}"] = bit_accuracy_
                metrics[f"pvalue_{label.lower().replace(' ', '_').replace('.', '')}"] = pvalue_
                metrics[f"capacity_{label.lower().replace(' ', '_').replace('.', '')}"] = capacity_
                # print(f"{label} - Bit Accuracy: {bit_accuracy_:.3f} - P-Value: {pvalue_:0.2e} - Capacity: {capacity_:.3f} - took {timer.stop():.2f}s")

        print(metrics)

        del vid, outputs, imgs, imgs_w, diff

Model loaded successfully from /checkpoint/pfz/2025_logs/0219_vseal_convnextextractor/_nbits=96_lambda_i=0.1_embedder_model=1/checkpoint400.pth with message: <All keys matched successfully>


Processing Videos for 96b_y_400:   0%|          | 0/1 [00:00<?, ?it/s]

loading video /checkpoint/pfz/projects/videoseal/assets/videos/metamoviegen_3s/01.mp4 - took 2.30s
embedding watermark  - took 1.47s
compressing and detecting watermarks
{'psnr': 40.01866149902344, 'ssim': 0.9879189729690552, 'msssim': 0.9936320185661316, 'linf': 24.331357955932617, 'bit_accuracy_original': 0.9895833134651184, 'pvalue_original': 1.2243121249030103e-27, 'capacity_original': 87.97987365722656, 'bit_accuracy_h264_30_+_crop_08': 0.5208333134651184, 'pvalue_h264_30_+_crop_08': 0.37982464416181483, 'capacity_h264_30_+_crop_08': 0.1202545166015625}


Processing Videos for 96b_y_400: 100%|██████████| 1/1 [00:47<00:00, 47.28s/it]


In [ ]:
outputs = wam.detect(imgs_aug, is_video=True)
outputs["preds"]

In [ ]:
wam = setup_model_from_checkpoint(ckpt)
wam.eval()
wam.to(device)

video_path = "/private/home/pfz/09-videoseal/baselines/outputs/videoseal0.4_01_wm_crop_08_h264_40.mp4"
vid, mask = VideoDataset.load_full_video_decord(video_path)

timer.start()
bit_preds = wam.detect_and_aggregate(imgs_aug)
bit_accuracy_ = bit_accuracy(
    bit_preds,
    msgs[:1]
).nanmean().item()
pvalue_ = pvalue(
    bit_preds,
    msgs[:1]
).nanmean().item()
capacity_ = capacity(
    bit_preds,
    msgs[:1]
).nanmean().item()
print(f"CRF={crf} - Bit Accuracy: {bit_accuracy_:.3f} - P-Value: {pvalue_:0.2e} - Capacity: {capacity_:.3f} - took {timer.stop():.2f}s")

In [ ]:
msgs

In [ ]:
bit_preds